In [1]:
import pandas as pd
from path import Path

In [3]:
track_cmc_merged_df = pd.read_csv(Path(f"../../../../data/cleandata/track_cmc_merged_df.csv"))

In [16]:
track_cmc_merged_df.shape

(1010, 21)

In [4]:
ticker_sentiments_df = pd.read_csv(Path(f"../../../../data/cleandata/ticker_sentiments.csv"))

In [15]:
ticker_sentiments_df.shape

(899, 6)

In [7]:
ticker_sentiments_df.rename(columns = {'ticker':'Ticker'}, inplace = True)

In [9]:
merged_inner = pd.merge(left=track_cmc_merged_df,right=ticker_sentiments_df, left_on='Ticker', right_on='Ticker')
merged_inner.head()

,rank_cnc,Name,Ticker,market_cap,price,volume24,pct1h,pct24h,pct7d,country,...,type,End,Start,Duration,pre_Duration,compound,negative,neutral,positive,no_of_posts
0,7,Binance Coin,BNB,"3,277,372,658",21.07,"231,846,713",-0.46,-1.33,2.49,Japan,...,ICO,2017-07-21 07:00:00,2017-07-01 07:00:00,20 days 00:00:00.000000000,0 days 00:00:00.000000000,0.563773,0.096670,0.041410,0.861890,100
1,8,EOS,EOS,"3,214,893,394",3.42,"1,735,988,258",0.11,-1.49,-2.55,Cayman Islands,...,ICO,2018-06-12 00:00:00,2017-06-26 16:00:00,350 days 08:00:00.000000000,0 days 00:00:00.000000000,0.828470,0.032200,0.858550,0.109270,100
2,13,Cardano,ADA,"1,105,888,653",0.042654,"53,437,224",-0.50,-1.72,-2.12,Switzerland,...,ICO,2017-01-01 00:00:00,2015-09-01 00:00:00,488 days 00:00:00.000000000,0 days 00:00:00.000000000,0.589474,0.098387,0.032946,0.868753,93
3,18,Tezos,XTZ,"821,746,518",1.24,"44,390,717",-0.25,10.13,2.47,United States of America,...,ICO,2017-07-14 12:00:00,2017-07-01 12:00:00,13 days 00:00:00.000000000,0 days 00:00:00.000000000,0.663939,0.021939,0.878857,0.099327,49
4,19,Cosmos,ATOM,"760,265,147",3.99,"147,439,544",-1.62,1.29,2.14,Canada,...,ICO,2017-04-06 13:00:00,2017-04-06 12:00:00,0 days 01:00:00.000000000,0 days 00:00:00.000000000,0.340496,0.060687,0.023224,0.916194,67


In [11]:
merged_inner.shape

(364, 26)

In [13]:
def check_what_matches(df1,df2):
    duplicates = []
    
    for track in df1.index.unique().tolist():
        for watch in df2.index.unique().tolist():
            if track == watch:
                duplicates.append(track)
            else:
                continue
    return duplicates
#len(check_what_matches(track_cmc_merged_df,ticker_sentiments_df))

In [14]:
file_path = Path("../../../../data/cleandata/sentiment_cmc_track_merge.csv")
merged_inner.to_csv(file_path, index=False)